# Використання моделі noobai-XL-1.1 у Google Colab

У цьому ноутбуці ми налаштуємо середовище для використання моделі `noobai-XL-1.1` з Hugging Face, завантажимо модель, та продемонструємо, як її використовувати для генерації тексту.

In [ ]:
# Встановлення необхідних бібліотек
!pip install transformers torch

## Імпорт необхідних модулів

Імпортуємо бібліотеки, необхідні для роботи з моделлю.

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

## Завантаження токенізатора та моделі

Завантажимо токенізатор і модель з Hugging Face. Переконайтесь, що ви маєте доступ до моделі `noobai-XL-1.1`.

In [ ]:
# Завантаження токенізатора
tokenizer = AutoTokenizer.from_pretrained("Laxhar/noobai-XL-1.1")

# Завантаження моделі
model = AutoModelForCausalLM.from_pretrained("Laxhar/noobai-XL-1.1")

# Перевірка доступності GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

## Використання моделі для генерації тексту

Нижче наведено приклад використання моделі для генерації тексту на основі заданого промпту. Ви можете налаштовувати параметри генерації відповідно до ваших потреб.

In [ ]:
# Введення користувача
prompt = "Напишіть вступ до наукової статті про штучний інтелект:"

# Токенізація введення
inputs = tokenizer.encode(prompt, return_tensors="pt").to(device)

# Генерація тексту
outputs = model.generate(
    inputs,
    max_length=200,          # Максимальна довжина генерованого тексту
    num_return_sequences=1,  # Кількість варіантів генерації
    no_repeat_ngram_size=2,  # Уникнення повторення n-грам
    temperature=0.7,         # Контроль креативності генерації
    top_p=0.9,               # Ядерне вибірка для генерації
    do_sample=True            # Використання семплінгу
)

# Декодування та вивід результату
generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(generated_text)

## Налаштування параметрів генерації

Ви можете експериментувати з різними параметрами генерації, щоб отримати бажані результати. Нижче наведено опис деяких ключових параметрів:

- `max_length`: Визначає максимальну довжину генерованого тексту.
- `num_return_sequences`: Кількість різних варіантів тексту, які генерує модель.
- `temperature`: Контролює випадковість генерації. Менші значення роблять текст більш передбачуваним.
- `top_p`: Використовується для ядерної вибірки, де `top_p=0.9` означає, що модель розглядає найімовірніші слова до тих пір, поки їх сума не досягне 90%.
- `no_repeat_ngram_size`: Забезпечує, що n-грам певної довжини не повторюється в генерованому тексті.
- `do_sample`: Визначає, чи використовувати семплінг (True) чи жорсткий пошук (False).

## Додаткові ресурси

- [Документація Transformers](https://huggingface.co/docs/transformers/index)
- [Приклади використання моделей Hugging Face в Colab](https://huggingface.co/transformers/installation.html#google-colaboratory)